# Lab 0 - Part II: Secrets of Happiness

In the first part of this lab, we learned how to use Python to run
different kinds of simulations, such as:

- generate a population with different characteristics,
- match individuals in the marriage market based on pre-defined criteria,
- generate the happiness score of each individual based on their characteristics.

At the end of last lab, we saved our data in a CSV file, called
`marriage_data.csv`. In this part, we will learn how to analyze the
data we generated with `R`.

Most of the time, you just need to run the code in the cells below by
clicking on the "Run" button. If you see `# Change the code below` in
the cell, you need to modify the code before running it.

In [1]:
# load libraries and some helper functions
library(pacman)
p_load(data.table, magrittr, ggplot2, knitr, psych, stargazer)

# color palette
gray_scale <- c('#F3F4F8','#D2D4DA', '#B3B5BD', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

ft_palette <- c('#990F3D', '#0D7680', '#0F5499', '#262A33', '#FFF1E5')

ft_contrast <- c('#F83', '#00A0DD', '#C00', '#006F9B', '#F2DFCE', '#FF7FAA',
                 '#00994D', '#593380')

peep_head <- function(dt, n = 5) {
    dt %>%
        head(n) %>%
        kable()
}

peep_sample <- function(dt, n = 5) {
    dt %>%
        .[sample(.N, n)] %>%
        kable()
}

peep_tail <- function(dt, n = 5) {
    dt %>%
        tail(n) %>%
        kable()
}

## First step of data analysis: Load the data

The first step of data analysis is to load the data. We will use the
`fread` function from the `data.table` package to load the data. The
`fread` means file read, and it is a very fast function to read data
from a file.

In [ ]:
# mdt means marriage data table
# you can name it whatever you want
mdt <- fread("../data/marriage_data.csv")

In [ ]:
# check the structure of the data
str(mdt)

In [ ]:
head(mdt)

In [ ]:
tail(mdt)

## What's the share of married people in the population?

From the first part of this lab, we know that people who are married
should have a `matched_id` different from `NA`. We can use this
information to calculate the share of married people in the
population.

In [ ]:
# how many people are single?
# is.na() function is used to check missing values
# .N is a special symbol in data.table to calculate the number of rows
mdt %>%
    .[is.na(matched_id), .N]

In [ ]:
# with means with the dataset
# table gives the frequency of each unique value
# prop.table gives the proportion of each unique value
mdt %>%
    with(table(is.na(matched_id))) %>%
    prop.table() %>%
    round(4)

As the above results show that we have around $88.04\%$ of married
people in the population and $11.96\%$ of single people.

## Distribution of Variables

Now, we want to check the distribution of the variables in the
population. We will try to find out whether the variables are normally
distributed or not.

In [ ]:
# let's check the age first
mdt %>%
    with(summary(age))

In [ ]:
mdt %>%
    .[, .(age)] %>%
    head()

In [ ]:
mdt %>%
    with(hist(age, breaks = 20))

It looks like the `age` variable follows the uniform distribution, which
means that the age of people in the population is uniformly
distributed.

In [ ]:
# now let's check the income
mdt %>%
    with(summary(income))

In [ ]:
mdt %>%
    with(hist(income, breaks = 20))

The distribution of the `income` variable is right-skewed, which means
that most people have low income, and only a few people have high
income. This is consistent with the real world. However the log
transformation of the `income` variable is normally distributed
as it is shown in the following figure.

In [ ]:
# do the log transformation
mdt %>%
    with(hist(log(income), breaks = 20))

In [ ]:
# put tow histograms side by side
options(repr.plot.width = 8, repr.plot.height = 5)
par(mfrow = c(1, 2))
hist(mdt$income, breaks = 20, main = "Income")
hist(log(mdt$income), breaks = 20, main = "Log Income")

In [ ]:
# distribution of openess
mdt %>%
    with(hist(openness, breaks = 20))

## Plot distribution of selected variables at one time

In [ ]:
str(mdt)

In [ ]:
# chagne figure size
options(repr.plot.width = 10, repr.plot.height = 10)
mdt %>%
    # select the columns you want to plot
    .[, .(age, income, openness, agreeableness, tolerance, happiness)] %>%
    pairs.panels(
        method = "pearson", # correlation method
        hist.col = "#00AFBB",
        density = TRUE,  # show density plots
    )

## Does marriage make people happy?

Now, we want to check whether married people are happier than single
people. We will first plot the distribution of the happiness score for
married and single people. And then we will compare the average
happiness score of married and single people. We will also run
statistical tests to check whether the difference in the average
happiness score is statistically significant or not.

In [ ]:
# add a new column to the data table
mdt %>%
    .[, is_married := ifelse(is.na(matched_id), 'No', 'Yes')] %>%
    head()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 7)
mdt %>%
    with(boxplot(happiness ~ is_married))

From the above plot, it seems that married people are less happy than
single people. Does this mean that marriage makes people unhappy?

In [ ]:
# let's run the t-test
# t.test is a function in R to run t-test
mdt %>%
    .[!is.na(happiness)] %>%
    with(t.test(happiness ~ is_married))

For this t-test, our hypothesis is:

- $H_0$: The average happiness score of married people is the same as
  the average happiness score of single people.
- $H_1$: The average happiness score of married people is different

Based on the t-test results, we can reject the null hypothesis and
conclude that the average happiness score of married people is
different from the average happiness score of single people.

## Selection Bias, reverse causality, and omitted variable bias

From the above results, we can see that married people are less happy
than single people. However, this does not mean that marriage makes
people unhappy. There are several reasons why married people are less
happy than single people:

- **Selection Bias**: People who are married may have different
  characteristics than people who are single. For example, married
  people may be older than single people, and older people may be less
  happy than younger people. When we generate the population, we
  did not use marriage as a factor to generate the happiness score.
- **Reverse Causality**: People who are less happy may be less likely
  to get married. For example, people who are less happy may be less
  likely to get married because they are less likely to find a
  partner. When we generate the population, we did not use the
  happiness score as a factor to generate marriage.
- **Omitted Variable Bias**: There may be other factors that affect
  both marriage and happiness. For example, people who are more
  open and agreeable may be more likely to get married and may be happier.

This is the formulat we used to generate the happiness score:

$$
\text{Happiness} = 0.03 \times \text{Income} + 0.4 \times \text{Openness} + 0.32 \times \text{Agreeableness} + 0.57 \times \text{Tolerance} +  0.02 \times (\text{Age} - 40)^2 
$$

> Takeaway: Correlation does not imply causation. Statistical tests
do not prove causation. When we find some statistical difference, we
need to think about the reasons behind it. Sometimes, the difference
is due to selection bias, reverse causality, or omitted variable bias.
Or it may be due to random chance.

## Your first regression analysis

Now, we will run a regression analysis to check whether marriage has a
causal effect on happiness. The regression analysis will help us to
control for other factors that may affect happiness, such as income,
age, openness, agreeableness, and tolerance, etc.

In this regression analysis, we will not use `openness`, `agreeableness`, or `tolerance` as those factors are not measured in the real world. We will only use `income`, `age`, and `married` as the independent variables.

In [ ]:
head(mdt)

In [ ]:
# run a linear regression
# lm is a function in R to run linear regression
lm_model1 <- lm(happiness ~ age + income + is_married, data = mdt)
summary(lm_model1)

In [ ]:
stargazer(lm_model1, type = "text")

In [ ]:
# using qudratic term for age
lm_model2 <- lm(happiness ~ age + I(age^2) + income + is_married, data = mdt)
summary(lm_model2)

In [ ]:
stargazer(lm_model2, type = "text")

As you can see that from the second regression model, the coefficient
for `married` is not statistically significant. This means that
marriage does not have effect on happiness. This is because we have
controlled for other factors that may affect happiness, such as income ang age.

When we only use linear term for `age` in the regression model, the coefficient for `married` is statistically significant. This means that finding the right regression model is very important. As the following graph shows that happiness score first
decreases and then increases with age. This is consistent with the
real world. This is also how we generated the happiness score in the
first part of this lab.

<iframe src="https://www.desmos.com/calculator/w79mztebg5?embed" width="500" height="500" style="border: 1px solid #ccc" frameborder=0></iframe>

In [ ]:
lm_model3 <- lm(happiness ~ age + I(age^2) + income + is_married + openness + agreeableness + tolerance, data = mdt)
summary(lm_model3)

In [ ]:
stargazer(lm_model3, type = "text")

Now, please check the coefficients of the regression model and compare 
them with the coefficients we used to generate the happiness score
in the following formula:

$$
\text{Happiness} = 0.03 \times \text{Income} + 0.4 \times \text{Openness} + 0.32 \times \text{Agreeableness} + 0.57 \times \text{Tolerance} +  0.02 \times (\text{Age} - 40)^2 
$$